In [ ]:
import pandas as pd
import numpy as np

# 1. Load data ONCE
df = pd.read_csv('/customer_signups.csv')

# === CLEANING ===

# Dates
df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce')

# Plan cleaning
df['plan_selected'] = df['plan_selected'].str.strip().str.lower()
mapping_plan = {
    'basic': 'Basic',
    'pro': 'Pro',
    'premium': 'Premium'
}
df['plan_selected'] = df['plan_selected'].map(mapping_plan)

# Gender cleaning
df['gender'] = df['gender'].str.strip().str.lower()
mapping_gender = {
    'male': 'Male',
    'm': 'Male',
    'female': 'Female',
    'f': 'Female',
    'other': 'Other',
    'non-binary': 'Other'
}
df['gender'] = df['gender'].map(mapping_gender)

# Remove duplicate customers
before = len(df)
df = df.drop_duplicates(subset='customer_id', keep='first')
after = len(df)
duplicates_removed = before - after
print("Duplicates removed:", duplicates_removed)

# Region / source cleaning
df['region'] = df['region'].fillna('Unknown')
df['source'] = df['source'].replace({'??': 'Unknown'})

# Age cleaning
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df.loc[(df['age'] < 18) | (df['age'] > 100), 'age'] = np.nan
median_age = df['age'].median()
df['age'] = df['age'].fillna(median_age)

# Drop rows with missing email
df = df.dropna(subset=['email'])

# Quick check
print("Plan selected values:", df['plan_selected'].unique())
print("Gender values:", df['gender'].unique())
print("Region values:", df['region'].unique())
print("Source values:", df['source'].unique())

# === SUMMARY OUTPUTS ON CLEANED DF ===

# Sign-ups per week
signups_per_week = (
    df.dropna(subset=['signup_date'])         # make sure no NaT in index
      .set_index('signup_date')
      .resample('W')['customer_id']
      .count()
)
print("\nSign-ups per week:")
print(signups_per_week)

# Sign-ups by source
signups_by_source = df['source'].value_counts(dropna=False)
print("\nSign-ups by source:")
print(signups_by_source)

# Sign-ups by region
signups_by_region = df['region'].value_counts(dropna=False)
print("\nSign-ups by region:")
print(signups_by_region)

# Sign-ups by plan_selected
signups_by_plan = df['plan_selected'].value_counts(dropna=False)
print("\nSign-ups by plan_selected:")
print(signups_by_plan)

# Marketing opt-in counts by gender
marketing_optin_by_gender = (
    df.groupby('gender')['marketing_opt_in']
      .value_counts(dropna=False)
)
print("\nMarketing opt-in by gender:")
print(marketing_optin_by_gender)

# Age summary (on cleaned ages)
age_summary = {
    'min': df['age'].min(),
    'max': df['age'].max(),
    'mean': df['age'].mean(),
    'median': df['age'].median(),
    'null_count': df['age'].isna().sum()
}
print("\nAge summary:")
print(age_summary)


Duplicates removed: 1
Plan selected values: ['Basic' 'Premium' 'Pro' nan]
Gender values: ['Male' 'Other' 'Female' nan]
Region values: ['West' 'North' 'Unknown' 'South' 'Central' 'East']
Source values: ['LinkedIn' 'Google' 'YouTube' 'Facebook' 'Referral' 'Instagram' nan
 'Unknown']

Sign-ups per week:
signup_date
2024-01-07    6
2024-01-14    5
2024-01-21    6
2024-01-28    6
2024-02-04    8
2024-02-11    5
2024-02-18    5
2024-02-25    6
2024-03-03    7
2024-03-10    7
2024-03-17    4
2024-03-24    5
2024-03-31    5
2024-04-07    6
2024-04-14    4
2024-04-21    6
2024-04-28    6
2024-05-05    6
2024-05-12    4
2024-05-19    6
2024-05-26    6
2024-06-02    6
2024-06-09    6
2024-06-16    4
2024-06-23    6
2024-06-30    6
2024-07-07    7
2024-07-14    5
2024-07-21    5
2024-07-28    6
2024-08-04    7
2024-08-11    4
2024-08-18    5
2024-08-25    6
2024-09-01    6
2024-09-08    6
2024-09-15    4
2024-09-22    6
2024-09-29    6
2024-10-06    5
2024-10-13    5
2024-10-20    5
2024-10-27    

/tmp/ipython-input-1836329566.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce')
